## Cookbook for using VectaraDB with Embedchain

[Vectara](https://vectara.com/) is the trusted GenAI and semantic search platform that provides an easy-to-use API for document indexing and querying. 

Vectara provides an end-to-end managed service for Retrieval Augmented Generation or [RAG](https://vectara.com/grounded-generation/), which includes:

1. A way to extract text from document files and chunk them into sentences.

2. The state-of-the-art [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) embeddings model. Each text chunk is encoded into a vector embedding using Boomerang, and stored in the Vectara internal knowledge (vector+text) store. Thus, when using Vectara with EmbedChain you do not need to call a separate embedder model - this happens automatically within the Vectara backend.

3. A query service that automatically encodes the query into embedding, and retrieves the most relevant text segments (including support for [Hybrid Search](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching) and [MMR](https://vectara.com/get-diverse-results-and-comprehensive-summaries-with-vectaras-mmr-reranker/))

4. An option to create [generative summary](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview), based on the retrieved documents, including citations.

See the [Vectara API documentation](https://docs.vectara.com/docs/) for more information on how to use the API.

This notebook demonstrates how to use Vectara with EmbedChain.

### Step-1: Install embedchain package

In [1]:
!pip install embedchain[vectara]

Obtaining file:///Users/ofer/dev/embedchain
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done


  Building editable for embedchain (pyproject.toml) ... done
  Created wheel for embedchain: filename=embedchain-0.1.64-py3-none-any.whl size=9350 sha256=ab9a9d64c2241676cfe5f43f8a63540d17b3d2d451bb6b090ca0f129c28dc331
  Stored in directory: /private/var/folders/48/8_815vxj0vl2z5_28gys249r0000gn/T/pip-ephem-wheel-cache-5fzfn0i5/wheels/c1/10/22/d1bfbf20a6f1021e5a5c8f177f263d6d6ba4ea62066fbf8671
Successfully built embedchain
  Attempting uninstall: embedchain
    Found existing installation: embedchain 0.1.64
    Uninstalling embedchain-0.1.64:
      Successfully uninstalled embedchain-0.1.64


### Step-2: Set environment variables needed for Vectara

You will need a Vectara account to use Vectara with EmbedChain. To get started, use the following steps:

1. [Sign up](https://vectara.com/integrations/embedchain) for a Vectara account if you don't already have one. Once you have completed your sign up you will have a Vectara customer ID. You can find your customer ID by clicking on your name, on the top-right of the Vectara console window.

2. By default you do not have to create a corpus, the EmbedChain integration with Vectara will create one for you (see "collection-name" in the configuration). If you want to pre-create that corpus, you can use the **"Create Corpus"** button. 

3. You will need to use Vectara's [OAUTH authentication](https://docs.vectara.com/docs/learn/authentication/oauth-2), and define the following environment variables:
* `VECTARA_CUSTOMER_ID`
* `VECTARA_OAUTH_CLIENT_ID`
* `VECTARA_OAUTH_SECRET`

If you are using Vectara as purely a retrieval engine (without summarization) then you would also need an OpenAI API key for the summarization

* `OPENAI_API_KEY`

You can have those environment variables set in the OS environment, or set these variables using os.environ as follows:

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["VECTARA_CUSTOMER_ID"] = "xxx"
os.environ["VECTARA_OAUTH_CLIENT_ID"] = "xxx"
os.environ["VECTARA_OAUTH_SECRET"] = "xxx"

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


### Step-3 Create embedchain app and define your config

Since Vectara provides a GenAI platform (we call it RAG-in-a-box) you do not need an embedding provider as this is done inside the platform. So you can just use the FakeEmbedder instead.
Furthermore, Vectara does its own chunking internally, so we configure the chunking to be a no-op (chunk size is 1M tokens):

In [3]:
from embedchain import App

ec_config = {
    "app": {
        "config": {
            "id": "vectara-docs",
        }
    },
    "vectordb": {
        "provider": "vectara",
        "config": {
            "collection_name": "vectara-demo",
        }
    },
    "chunker": {
        "chunk_size": 1000000,
        "chunk_overlap": 0
    },
    "embedder": {
        "provider": "fake",
    }
}

And as usual, we start by creating the EmbedChain app:

In [4]:
app = App.from_config(config=ec_config)

### Step-4: Add data sources to your app

For our example we will add the full content of Vectara's documentation site (choosing the "docs_site" data type).

Note that specifying the `app.config.id` in the configuration above creates a unique APP_ID, ensuring that data already indexed will not be re-indexed in Vectara.

In [5]:
app.add("https://docs.vectara.com/", data_type="docs_site")

Adding 0 new documents to Vectara (out of 42 submitted to indexing)...
Successfully saved https://docs.vectara.com/ (DataType.DOCS_SITE). New chunks count: 0


'd1f20d25579de0cf550ce26446eaeaaa'

### Step-5: Let's ask some questions using Vectara as a vector DB

In [6]:
questions = [
    "What is Vectara?",
    "How do I index documents in Vectara?",
    "what is hybrid search and how does it work in Vectara?",
    "What is RAG?",
    "How does MMR work?"
]

data = []
for q in questions:
    answer = app.query(q)
    data.append([q, answer])

df = pd.DataFrame(data=data, columns=["question", "answer"])

In [7]:
df.style.set_properties(**{'text-align': 'left'})

,question,answer
0,What is Vectara?,"Vectara is a platform that allows users to create and manage corpora, which are collections of documents. It provides various APIs for indexing and segmenting documents, as well as features for deleting and uploading files. Vectara is flexible in terms of the types of documents it can handle, ranging from short tweets to lengthy texts like the Bible. Indexing data into Vectara is typically a fast process, taking only a few seconds."
1,How do I index documents in Vectara?,"To index documents in Vectara, you can use either the Standard Indexing API or the File Upload API. The Standard Indexing method is recommended for indexing a set of semi-structured documents or content into a corpus. Vectara will index and chunk the documents for you. The File Upload method allows you to upload and index files into a corpus using an HTTP endpoint. The first step in using Vectara is to index a set of related documents or content into a corpus. Indexing a document makes the data available for search and retrieval more efficiently."
2,what is hybrid search and how does it work in Vectara?,"Hybrid search in Vectara is a retrieval model that combines traditional, keyword-based search with semantic search. It blends partial, exact, and Boolean text matching with neural models to provide a powerful and flexible approach to text retrieval. In Vectara, hybrid search allows you to include exact keyword matches for search terms that were not present in Vectara's training data, such as product SKUs. It also enables you to disable neural retrieval entirely and instead use exact term matching. Additionally, you can incorporate typical keyword modifiers like a NOT function, exact phrase matching, and wildcard prefixes of terms. To enable hybrid search in Vectara, you need to specify a value called ""lambda"" at query time, specifically under the corpusKey. The lambda value can range from 0 to 1 (inclusive)."
3,What is RAG?,"RAG stands for Retrieval Augmented Generation. It is a technology used by Vectara to ensure that generated content is both verifiable and anchored to the data provided. RAG minimizes the occurrence of hallucinations, which are inaccurate or misleading information commonly found in generative AI systems. It summarizes search results that answer complex queries directly while providing citations that ground these search results in facts from the data."
4,How does MMR work?,I don't have enough information to answer the query.


And now with the citations:

In [8]:
app.query("What is Vectara?", citations=True)

('Vectara is a platform that allows users to create and manage corpora, which are collections of documents. It provides various APIs for indexing and segmenting documents, as well as features for deleting and uploading files. Vectara is flexible in terms of the types of documents it can handle, ranging from short tweets to lengthy texts like the Bible. Indexing data into Vectara is typically a fast process, taking only a few seconds.',
 [("Why certain birds are allowed for the most random reason? We have part of a\nunique employee handbook ready for you to upload into a Vectara corpus,\nand we'll guide you through the ingestion and question answering process step by step. <%START%>Step 1. Create a Vectara account \u200b To get started with Vectara , go to https://console.vectara.com/signup or\nclick Get Started Free at vectara.com.<%END%> After you make an account, you can\ncreate your first corpus and upload your first document! Step 2. Create your first corpus \u200b Before you can a

### Step-6: Now use Vectara's full end-to-end RAG capabilities:
Note that in this case we use the `VectaraApp` class which makes sure the LLM is called from the Vectara backend and you do not need to call OpenAI or another LLM for the summarization, instead this comes from the Vectara service.

In [9]:
from embedchain.vectordb.vectara import VectaraApp
app = VectaraApp.from_config(config=ec_config)

In [10]:
questions = [
    "What is Vectara?",
    "How do I index documents in Vectara?",
    "what is hybrid search and how does it work in Vectara?",
    "What is RAG?",
    "How does MMR work?"
]

data = []
for q in questions:
    answer = app.query(q)
    data.append([q, answer])

df = pd.DataFrame(data=data, columns=["question", "answer"])

In [11]:
df.style.set_properties(**{'text-align': 'left'})

,question,answer
0,What is Vectara?,"Vectara is a platform designed for developers with an API-first approach. It offers a complete end-to-end solution for integrating generative AI search into applications. Vectara allows users to create a corpus, upload and index documents, and extract text and metadata from files such as PDFs and Word Documents. With Vectara, you can create an answer engine and benefit from its hybrid search core and superior language understanding capabilities. The platform provides easy ingestion, simple APIs, and fine-grained control over indexing and segmentation. It is a secure and developer-focused solution for integrating AI search functionality ."
1,How do I index documents in Vectara?,"The Vectara platform offers different methods for indexing documents. The recommended approach is to use the Standard Indexing API, which is suitable for structured documents and allows Vectara to index and chunk the content efficiently. Another option is to utilize the File Upload API, which provides an HTTP endpoint to upload and index files into a corpus. Additionally, Vectara supports filter expressions to restrict searches to specific parts of the corpus based on metadata. To manage the ingested documents and check their status, you can use the ListDocuments API. Please note that the provided search results did not contain sufficient information to be summarized into a comprehensive answer."
2,what is hybrid search and how does it work in Vectara?,"Hybrid search in Vectara combines partial, exact, and Boolean text matching with neural models, blending traditional keyword-based search with semantic search. This approach allows for powerful and flexible text retrieval. Vectara deploys advanced zero-shot models and conversational search capabilities to understand and interpret user queries accurately. It offers a hybrid retrieval model that includes exact keyword matches, disabling neural retrieval, and incorporating keyword modifiers. Vectara's hybrid search can be enabled by specifying a value between 0 and 1 at query time. This approach ensures that Vectara provides precise and contextually accurate responses, making it an optimal choice for integrating generative AI search into applications ."
3,What is RAG?,"Retrieval Augmented Generation (RAG) is a technique used by Vectara to ensure that generated content is verifiable and anchored to the provided data, reducing the occurrence of hallucinations in AI systems. RAG combines retrieval-based methods with generative models to answer complex queries directly and provide citations that ground the search results in factual data. By minimizing hallucinations, Vectara enhances trust in AI-powered decision making. This approach is language-agnostic, allowing users to search across multiple languages. RAG can be used to establish question-answer databases, providing authoritative answers to users. It addresses the problem of false information by focusing on facts and reducing inaccurate and misleading responses. Vectara offers an API Playground for developers to experiment with their REST APIs and understand the fundamentals of APIs. Vectara also provides demo applications that showcase the power of GenAI conversational search and RAG ."
4,How does MMR work?,The returned results did not contain sufficient information to be summarized into a useful answer for your query. Please try a different search or restate your query differently.


And with citations enabled:

In [12]:
app.query("What is Vectara?", citations=True)

('Vectara is a platform that offers an API-first approach for developers to integrate generative AI search into their applications. It provides a hybrid search core and language understanding for ingestion and retrieval [4]. Vectara allows users to create a corpus, upload documents, and index them [2]. It supports various indexing operations, including creating, deleting, and indexing documents [3]. The platform is designed to be developer-focused, secure, and offers easy ingestion and simple APIs [4]. Additionally, Vectara provides a File Upload API for uploading and indexing files like PDFs and Word Documents [5]. Overall, Vectara aims to be an answer engine that enables efficient search and retrieval of information [4].',
 [("Why certain birds are allowed for the most random reason? We have part of a\nunique employee handbook ready for you to upload into a Vectara corpus,\nand we'll guide you through the ingestion and question answering process step by step. <%START%>Step 1. Create 